# Scraping sitio "Books To Scrape"
### https://books.toscrape.com/

### - Imports

In [119]:
from bs4 import BeautifulSoup
import requests

### - Variables globales

In [120]:

link_to_scrape = 'https://books.toscrape.com/'

### - Manejar recursos

In [127]:
class HTML():
    def get_text(self, link:str) -> str:
        try:
            return requests.get(link).text
        except requests.exceptions.MissingSchema as e:
            print(e)

class Parser():
    def parser(self, html):
        return BeautifulSoup(html, 'html.parser')

class ScrapingBooksToScrape():
    def __init__(self, link:str, recurso:HTML, parser:Parser) -> None:
        self._link_principal = link
        self._recurso_html = recurso
        self._parser = parser
        self._lista_links_principal = []
        self._lista_links_libros = []


    def busqueda_link_principal(self, classname:str) -> None:
        recurso = self._recurso_html.get_text(self._link_principal)

        if recurso:
            html = self._parser.parser(recurso)
            html_categorias = html(class_=classname)

            for categoria in html_categorias:
                enlaces = categoria.find_all('a')
                for link in enlaces:
                    self._lista_links_principal.append({
                        'categoria': link.text.strip(),
                        'link': link.get('href')
            })

    def busqueda_links_libros(self) -> None:
        print(len(self._lista_links_principal))
        for i, link in enumerate(self._lista_links_principal):
            full_url = self._link_principal + '/' + link['link']

            recurso = self._recurso_html.get_text(full_url)

            if recurso:
                html = self._parser.parser(recurso)

                tags_ol = html.find('ol', class_='row')

                for contenido in tags_ol:
                    print(type(contenido))

                # etiqueta_hijo = html.find('div', class_='page-header')
                # etiqueta_padre = etiqueta_hijo.find_parent()

                # tags_ol_class_row = etiqueta_padre.find_all('ol', class_='row')
                # tags_li_hijo_tag_ol_class_row = tags_ol_class_row[0].find('li')

                # print(self._lista_links_principal[i]['categoria'], ': ', tags_li_hijo_tag_ol_class_row.find('a').get('href'))

                # for tag_li in tags_li_hijo_tag_ol_class_row:
                #     tag_li.find('a')
                


                # self._lista_links_libros.append(html)

    def get_lista_links_principal(self) -> list[dict]:
        print(self._link_principal)
        return self._lista_links_principal
    
    def get_lista_links_libros(self):
        return self._lista_links_libros
    

        

spider = ScrapingBooksToScrape(link_to_scrape, HTML(), Parser())
spider.busqueda_link_principal('side_categories')
spider.busqueda_links_libros()

spider.get_lista_links_principal()
spider.get_lista_links_libros()

51
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>
<class 'bs4.element.Tag'>
<class 'bs4.element.NavigableString'>

[]